
<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesor: Víctor Navarro Aránguiz</em><br>

</div>

# Python para Data Science: Proyecto Final

**Fecha de Entrega**: 06 de Agosto de 2024

**Objetivos:**
  - Aplicar los conceptos aprendidos en clases.
  - Realizar el análisis de un dataset de su preferencia.
  - Familiarizarse con el manejo de versiones a través de Git

**Puntaje Total:** 24 Puntos

**Nombre Estudiante(s)**: `Jeanette León Vejar`

**Link Github**: [GitHub Proyecto Final](https://github.com/JeanetteLeon/Python_Proyecto_Final)

## Limpieza de Datos

#### Tratamiento de nulos:

- Hemos encontrado que en el campo `Metascore` tiene 2.007 nulos, equivalentes al 20% de universo del dataset, sin embargo es pausible esta evaluación tenga datos nulos, ya que se calcula a partir de las reseñas de películas hechas por un gran grupo de críticos más respetados en esta industria. Entonces pueden existir peliculas que no tengas estas criticas a gran escala y que sigan siendo populares en la audiencia.

Como es un dato numérico se tratará reemplazando los nulos por ceros y como se visualiza en gráfico boxplot.

#### Tratamiento de duplicados:

Como explicamos en el apartado anterior de análisis exploratorio, los duplicados que existen son posible y no son anomalías:
- Se halló que `ID` y `Link` no tiene duplicados, lo que está bien ya que son valores únicos para una película en este dataset.

- Los campos `Rating`, `Runtime`, `Genre`, `Metascore`, `Directors`, `Stars`, `Votes` y `Gross` tienen duplicados, ya que pueden repetirse en más de una película.

- Los campos `Movie Name` y `Plot` si tienen duplicados, ya que hacen referencia a peliculas diferentes como el caso los remakes, y tienen distintos directores y actores.

En conclusión no generará tratamiento para los duplicados.

#### Tratamiento de outliers y definición de nuevas variables:

Antes de evaluar el tratamiento de outliers, se modificarán la siguiente columna:

- El campo de `Runtime` se convertirá de string a integer, ya es un dato numérico.

Luego de revisar los outliers con boxplots y histogramas, se tomará como medida de tratamiento de outliers alternativo, la creación de nuevas variables para quitar el ruido del dataset.

- `ID` (integer): No tiene outliers.
- `Rating` (float): Tiene outliers pero están dentro del intervalo de 0 a 10.
- `Runtime` (integer): Tiene outliers, se puede dar dado el contexto que son películas y pueden haber largometrajes, en este dataset el intervalo de distribución está entre 45 minutos y 7 horas. Se creará una nueva variable `Largometraje`  donde puede ser si/no (booleano), para que quitar el ruido de este campo.
- `Metascore` (float): Tiene outliers pero están dentro del intervalo de 0 a 100. Y hay imputar los nulos como ceros se creó outliers, por tanto también se creará una columna `categ_metascore`, la cual podra ser si/no, donde no es igual cero (en el pasado fue nulo).
- `Votes` (integer): Tiene outliers extremos, dado el contexto se puede dar que peliculas tenga un gran número de votos. Por lo mismo se creará una variable llamada `Popularidad`,donde estarán 3 categorías: baja (0-100k votos), media (100k a 200k votos) y alta (>200k votos).
- `Gross` (integer): Tiene outliers extremos, dado el contexto se puede dar que peliculas que pueden generar un gran recaudación de ganancias (se revisó los casos extremos y se constato que son correctos). Por lo mismo se creará una variable llamada `Categ_ganancia`,donde estarán 3 categorías: baja (0-20M), media (20M a 100M votos) y alta (>100M).








In [143]:
# Librerías

import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt


In [144]:
# Se guarda en una varible el dataset de análisis
dataset = pd.read_csv('../data/Top_10000_Movies_IMDb.csv')
dataset

,ID,Movie Name,Rating,Runtime,Genre,Metascore,Plot,Directors,Stars,Votes,Gross,Link
0,1,The Shawshank Redemption,9.3,142 min,Drama,82.0,"Over the course of several years, two convicts...","['Frank Darabont', 'Tim Robbins', 'Morgan Free...","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",2752419,28341469,https://www.imdb.com/title/tt0111161/
1,2,The Godfather,9.2,175 min,"Crime, Drama",100.0,"Don Vito Corleone, head of a mafia family, dec...","['Francis Ford Coppola', 'Marlon Brando', 'Al ...","['Marlon Brando', 'Al Pacino', 'James Caan', '...",1914751,134966411,https://www.imdb.com/title/tt0068646/
2,3,Ramayana: The Legend of Prince Rama,9.2,135 min,"Animation, Action, Adventure",NaN,An anime adaptation of the Hindu epic the Rama...,"['Ram Mohan', 'Yûgô Sakô', 'Koichi Saski', 'Ar...","['Yûgô Sakô', 'Koichi Saski', 'Arun Govil', 'N...",10108,10108,https://www.imdb.com/title/tt0259534/
3,4,The Chaos Class,9.2,87 min,"Comedy, Drama",NaN,"Lazy, uneducated students share a very close b...","['Ertem Egilmez', 'Kemal Sunal', 'Münir Özkul'...","['Kemal Sunal', 'Münir Özkul', 'Halit Akçatepe...",41785,41785,https://www.imdb.com/title/tt0252487/
4,5,Daman,9.1,121 min,"Adventure, Drama",NaN,"The film is set in 2015. Sid, is a young docto...","['Lenka Debiprasad', 'Vishal Mourya', 'Karan K...","['Vishal Mourya', 'Karan Kandhapan', 'Babushan...",13303,13303,https://www.imdb.com/title/tt17592606/
...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9995,Yogi Bear,4.6,80 min,"Animation, Adventure, Comedy",35.0,A documentary filmmaker travels to Jellystone ...,"['Eric Brevig', 'Dan Aykroyd', 'Justin Timberl...","['Dan Aykroyd', 'Justin Timberlake', 'Anna Far...",22762,100246011,https://www.imdb.com/title/tt1302067/
9995,9996,Troll,4.6,82 min,"Comedy, Fantasy, Horror",30.0,A wicked troll king in search of a mystical ri...,"['John Carl Buechler', 'Michael Moriarty', 'Sh...","['Michael Moriarty', 'Shelley Hack', 'Noah Hat...",10948,5450815,https://www.imdb.com/title/tt0092115/
9996,9997,Paranormal Activity 4,4.6,88 min,"Horror, Mystery",40.0,It has been five years since the disappearance...,"['Henry Joost', 'Ariel Schulman', 'Stephen Dun...","['Ariel Schulman', 'Stephen Dunham', 'Katie Fe...",68430,53921300,https://www.imdb.com/title/tt2109184/
9997,9998,Dark Crimes,4.6,92 min,"Crime, Drama, Mystery",24.0,A murder investigation of a slain businessman ...,"['Alexandros Avranas', 'Jim Carrey', 'Marton C...","['Jim Carrey', 'Marton Csokas', 'Charlotte Gai...",19098,19098,https://www.imdb.com/title/tt1901024/


In [145]:
# Revisión de nulos
print(dataset.isnull().sum())


ID               0
Movie Name       0
Rating           0
Runtime          0
Genre            0
Metascore     2007
Plot             0
Directors        0
Stars            0
Votes            0
Gross            0
Link             0
dtype: int64


In [146]:

def tratamiento_nulos(dataset):
    # Reemplazar los nulos por ceros 
    dataset['Metascore'] = dataset['Metascore'].fillna(0)

    # Convertir de flotante a entero el campo de Metascore
    dataset['Metascore'] =  dataset['Metascore'].astype(int)
    
    return dataset

nuevo_dataset = tratamiento_nulos(dataset)
nuevo_dataset

,ID,Movie Name,Rating,Runtime,Genre,Metascore,Plot,Directors,Stars,Votes,Gross,Link
0,1,The Shawshank Redemption,9.3,142 min,Drama,82,"Over the course of several years, two convicts...","['Frank Darabont', 'Tim Robbins', 'Morgan Free...","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",2752419,28341469,https://www.imdb.com/title/tt0111161/
1,2,The Godfather,9.2,175 min,"Crime, Drama",100,"Don Vito Corleone, head of a mafia family, dec...","['Francis Ford Coppola', 'Marlon Brando', 'Al ...","['Marlon Brando', 'Al Pacino', 'James Caan', '...",1914751,134966411,https://www.imdb.com/title/tt0068646/
2,3,Ramayana: The Legend of Prince Rama,9.2,135 min,"Animation, Action, Adventure",0,An anime adaptation of the Hindu epic the Rama...,"['Ram Mohan', 'Yûgô Sakô', 'Koichi Saski', 'Ar...","['Yûgô Sakô', 'Koichi Saski', 'Arun Govil', 'N...",10108,10108,https://www.imdb.com/title/tt0259534/
3,4,The Chaos Class,9.2,87 min,"Comedy, Drama",0,"Lazy, uneducated students share a very close b...","['Ertem Egilmez', 'Kemal Sunal', 'Münir Özkul'...","['Kemal Sunal', 'Münir Özkul', 'Halit Akçatepe...",41785,41785,https://www.imdb.com/title/tt0252487/
4,5,Daman,9.1,121 min,"Adventure, Drama",0,"The film is set in 2015. Sid, is a young docto...","['Lenka Debiprasad', 'Vishal Mourya', 'Karan K...","['Vishal Mourya', 'Karan Kandhapan', 'Babushan...",13303,13303,https://www.imdb.com/title/tt17592606/
...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9995,Yogi Bear,4.6,80 min,"Animation, Adventure, Comedy",35,A documentary filmmaker travels to Jellystone ...,"['Eric Brevig', 'Dan Aykroyd', 'Justin Timberl...","['Dan Aykroyd', 'Justin Timberlake', 'Anna Far...",22762,100246011,https://www.imdb.com/title/tt1302067/
9995,9996,Troll,4.6,82 min,"Comedy, Fantasy, Horror",30,A wicked troll king in search of a mystical ri...,"['John Carl Buechler', 'Michael Moriarty', 'Sh...","['Michael Moriarty', 'Shelley Hack', 'Noah Hat...",10948,5450815,https://www.imdb.com/title/tt0092115/
9996,9997,Paranormal Activity 4,4.6,88 min,"Horror, Mystery",40,It has been five years since the disappearance...,"['Henry Joost', 'Ariel Schulman', 'Stephen Dun...","['Ariel Schulman', 'Stephen Dunham', 'Katie Fe...",68430,53921300,https://www.imdb.com/title/tt2109184/
9997,9998,Dark Crimes,4.6,92 min,"Crime, Drama, Mystery",24,A murder investigation of a slain businessman ...,"['Alexandros Avranas', 'Jim Carrey', 'Marton C...","['Jim Carrey', 'Marton Csokas', 'Charlotte Gai...",19098,19098,https://www.imdb.com/title/tt1901024/


In [147]:
# Creación de boxplot del campo Metascore con tratamiento de nulos
fig = px.box(nuevo_dataset, y='Metascore', title='Boxplot de Metascore con reemplazo de nulos por ceros')

# Mostrar boxplot
fig.show()

In [148]:
def modificacion_campos(nuevo_dataset):
    df = nuevo_dataset
    
    # Quitar los 4 últimos caracteres de la columna runtime
    df['Runtime'] = df['Runtime'].str[:-4]
    
    # Convertir de string a entero 
    df['Runtime'] =  df['Runtime'].astype(int)
    
    return df

nuevo_dataset =  modificacion_campos(nuevo_dataset)
nuevo_dataset
    

,ID,Movie Name,Rating,Runtime,Genre,Metascore,Plot,Directors,Stars,Votes,Gross,Link
2750,2751,Ala Vaikunthapurramuloo,7.3,163,"Action, Comedy, Drama",0,Fate plays a vital role in connecting the life...,"['Trivikram Srinivas', 'Allu Arjun', 'Pooja He...","['Allu Arjun', 'Pooja Hegde', 'Tabu', 'Jayaram']",21660,7,https://www.imdb.com/title/tt9537292/
9471,9472,Meeting Evil,5.3,89,"Crime, Drama, Mystery",0,John is taken on a murder-fueled ride by a mys...,"['Chris Fisher', 'Luke Wilson', 'Samuel L. Jac...","['Luke Wilson', 'Samuel L. Jackson', 'Leslie B...",12451,181,https://www.imdb.com/title/tt1810697/
9774,9775,Apartment 143,5.0,80,"Horror, Thriller",0,A team of parapsychologists try to figure out ...,"['Carles Torrens', 'Kai Lennox', 'Gia Mantegna...","['Kai Lennox', 'Gia Mantegna', ""Michael O'Keef...",11620,256,https://www.imdb.com/title/tt1757742/
8575,8576,I Spit on Your Grave 2,5.7,106,"Horror, Thriller",0,When Katie innocently accepts an offer to have...,"['Steven R. Monroe', 'Jemma Dallender', 'Joe A...","['Jemma Dallender', 'Joe Absolom', 'Yavor Baha...",31207,441,https://www.imdb.com/title/tt2537176/
7601,7602,The Decoy Bride,6.1,89,"Comedy, Drama, Romance",42,When the world's media descend on the remote S...,"['Sheree Folkson', 'Kelly Macdonald', 'David T...","['Kelly Macdonald', 'David Tennant', 'Alice Ev...",12034,542,https://www.imdb.com/title/tt1657299/
...,...,...,...,...,...,...,...,...,...,...,...,...
152,153,Top Gun: Maverick,8.3,130,"Action, Drama",78,"After thirty years, Maverick is still pushing ...","['Joseph Kosinski', 'Tom Cruise', 'Jennifer Co...","['Tom Cruise', 'Jennifer Connelly', 'Miles Tel...",596202,718732821,https://www.imdb.com/title/tt1745960/
676,677,Avatar,7.9,162,"Action, Adventure, Fantasy",83,A paraplegic Marine dispatched to the moon Pan...,"['James Cameron', 'Sam Worthington', 'Zoe Sald...","['Sam Worthington', 'Zoe Saldana', 'Sigourney ...",1345064,760507625,https://www.imdb.com/title/tt0499549/
229,230,Spider-Man: No Way Home,8.2,148,"Action, Adventure, Fantasy",71,"With Spider-Man's identity now revealed, Peter...","['Jon Watts', 'Tom Holland', 'Zendaya', 'Bened...","['Tom Holland', 'Zendaya', 'Benedict Cumberbat...",802644,804747988,https://www.imdb.com/title/tt10872600/
108,109,Avengers: Endgame,8.4,181,"Action, Adventure, Drama",78,After the devastating events of Avengers: Infi...,"['Anthony Russo', 'Joe Russo', 'Robert Downey ...","['Joe Russo', 'Robert Downey Jr.', 'Chris Evan...",1184770,858373000,https://www.imdb.com/title/tt4154796/


In [149]:
# Revisión de outliers de datos númericos

nuevo_dataset.describe().round(0)

,ID,Rating,Runtime,Metascore,Votes,Gross
count,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
mean,5000.0,7.0,110.0,47.0,91271.0,28740668.0
std,2887.0,1.0,22.0,28.0,168451.0,59719328.0
min,1.0,5.0,45.0,0.0,10001.0,7.0
25%,2500.0,6.0,96.0,31.0,16785.0,36541.0
50%,5000.0,7.0,106.0,53.0,33912.0,4244155.0
75%,7500.0,7.0,120.0,68.0,89656.0,33062950.0
max,9999.0,9.0,439.0,100.0,2752419.0,936662225.0


In [150]:
# Select only numeric columns
numeric_columns = nuevo_dataset.select_dtypes(include='number').columns

# Create histograms for each numeric column
for column in numeric_columns:
    fig = px.histogram(nuevo_dataset, x=column, title=f'Histograma de: {column}', nbins=50)
    fig.show()

In [153]:
# Seleccionar sólo columnas númericas
numeric_columns = nuevo_dataset.select_dtypes(include='number').columns

# Crear un boxplot para cada columna númerica del dataset
for column in numeric_columns:
    fig = px.box(nuevo_dataset, y=column, title=f'Boxplot de cada campo númerico: {column}')
    fig.show()